In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/aitormentxaka/Desktop/Data science/proyectos/Modelo machine learning regresión/src')
from utils import functions as fu 

### Limpieza datos test


1. Datos faltantes
2. Discretización de variables 
3. Guardar datos limpios

In [26]:
test_data= pd.read_csv("/Users/aitormentxaka/Desktop/Data science/proyectos/Modelo machine learning regresión/src/data/house-prices-advanced-regression-techniques/test.csv")
test_data.set_index("Id",inplace=True)

### 1. Datos faltantes 

In [27]:
#Cambiamos los Nan por "None"
columnas_sotano = ["BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2"]
for column in columnas_sotano:
   test_data[column].fillna("None", inplace=True)
    
#Cambiamos los valores Nan de las columnas del garaje "None"
columnas_garaje = ["GarageType","GarageFinish","GarageQual","GarageCond"]
for column in columnas_garaje:
   test_data[column].fillna("None", inplace=True)

#Como el año de construcción es de tipo float, lo sustituimos por 0. 
test_data["GarageYrBlt"].fillna(0, inplace=True)

#Sustituimos por None
test_data["FireplaceQu"].fillna("None", inplace=True)

#Sustituimos por None
test_data["MasVnrType"].fillna("None", inplace=True)

#Lo cambiamos por 0 
test_data["MasVnrArea"].fillna(0, inplace=True)

#Sustituimos por None
test_data["MiscFeature"].fillna("None", inplace=True)

# Los Nan en estas variables significan ausencia de piscina, de accceso al callejon y ausencia de valla. Cambiamos las tres por None 
columnas_faltantes = ["PoolQC","Alley","Fence"]
for column in columnas_faltantes:
   test_data[column].fillna("None", inplace=True)

#Cambiamos los Nan de la variable numérica por su media 
test_data["LotFrontage"].fillna(test_data["LotFrontage"].mean(), inplace=True)

#Las varibales categóricas las cambiamos por su moda 
columns_cat = ["MSZoning", "Electrical", "Utilities", "Functional", "SaleType"]
for column in columns_cat:
   test_data[column].fillna(test_data[column].mode()[0], inplace=True)

- Comprobamos si tenemos algún Nan más

In [28]:
valores_nan =test_data.isnull().sum().sum()
print("Número total de NaN en todo el DataFrame:", valores_nan)

Número total de NaN en todo el DataFrame: 13


In [29]:

test_data[test_data.columns[test_data.isna().any()]]

,Exterior1st,Exterior2nd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,KitchenQual,GarageCars,GarageArea
Id,,,,,,,,,,,
1461,VinylSd,VinylSd,468.0,144.0,270.0,882.0,0.0,0.0,TA,1.0,730.0
1462,Wd Sdng,Wd Sdng,923.0,0.0,406.0,1329.0,0.0,0.0,Gd,1.0,312.0
1463,VinylSd,VinylSd,791.0,0.0,137.0,928.0,0.0,0.0,TA,2.0,482.0
1464,VinylSd,VinylSd,602.0,0.0,324.0,926.0,0.0,0.0,Gd,2.0,470.0
1465,HdBoard,HdBoard,263.0,0.0,1017.0,1280.0,0.0,0.0,Gd,2.0,506.0
...,...,...,...,...,...,...,...,...,...,...,...
2915,CemntBd,CmentBd,0.0,0.0,546.0,546.0,0.0,0.0,TA,0.0,0.0
2916,CemntBd,CmentBd,252.0,0.0,294.0,546.0,0.0,0.0,TA,1.0,286.0
2917,VinylSd,VinylSd,1224.0,0.0,0.0,1224.0,1.0,0.0,TA,2.0,576.0


In [30]:
test_data["Exterior1st"].fillna(test_data["Exterior1st"].mode()[0], inplace=True)
test_data["Exterior2nd"].fillna(test_data["Exterior2nd"].mode()[0], inplace=True)
test_data["KitchenQual"].fillna(test_data["KitchenQual"].mode()[0], inplace=True)

In [31]:

columnas_a_llenar = ["BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "BsmtFullBath", "BsmtHalfBath", "GarageCars", "GarageArea"]
test_data.loc[:, columnas_a_llenar] = test_data.loc[:, columnas_a_llenar].fillna(0)


- Volvemos a comprobar si tenemos algún dato faltante

In [32]:
valores_nan =test_data.isnull().sum().sum()
print("Número total de NaN en todo el DataFrame:", valores_nan)

Número total de NaN en todo el DataFrame: 0


### 2. Discretización de variables

In [33]:
diccionario_BsmtCond = {"None": 0, "Po":0, "Fa":1, "TA": 2, "Gd": 3, "Ex": 4}
test_data['BsmtCond'] = test_data['BsmtCond'].replace(diccionario_BsmtCond)

diccionario_BsmtExposure = {"None": 0, "Mn": 1, "Av": 2, "Gd": 3, "No": 0}
test_data['BsmtExposure'] = test_data['BsmtExposure'].replace(diccionario_BsmtExposure)

diccionario_BsmtFinType1 = {"None": 0, "Unf": 4, "LwQ": 1, "Rec": 2, "BLQ": 3, "ALQ": 5, "GLQ": 6}
test_data['BsmtFinType1'] = test_data['BsmtFinType1'].replace(diccionario_BsmtFinType1)

diccionario_BsmtFinType2 = { "None":0, "Unf": 4, "LwQ": 1, "Rec": 2, "BLQ": 3, "ALQ": 5, "GLQ": 6}
test_data['BsmtFinType2'] = test_data['BsmtFinType2'].replace(diccionario_BsmtFinType2)

diccionario_BsmtQual = {"None": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4}
test_data['BsmtQual'] = test_data['BsmtQual'].replace(diccionario_BsmtQual)


dic_Alley = {"Grvl": 2, "Pave": 1, "None": 0}
test_data['Alley'] = test_data['Alley'].replace(dic_Alley)

dic_ExterCond = {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 6}
test_data['ExterCond'] = test_data['ExterCond'].replace(dic_ExterCond)

dic_ExterQual = {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}
test_data['ExterQual'] = test_data['ExterQual'].replace(dic_ExterQual)

dic_GarageCond = {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}
test_data['GarageCond'] = test_data['GarageCond'].replace(dic_GarageCond)

dic_GarageQual = {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}
test_data['GarageQual'] = test_data['GarageQual'].replace(dic_GarageQual)

dic_PoolQC = {"None": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 8}
test_data['PoolQC'] = test_data['PoolQC'].replace(dic_PoolQC)

dic_Functional = {"Sal": 1, "Sev": 2, "Maj2": 3, "Maj1": 4, "Mod": 5, "Min2": 6, "Min1": 7, "Typ": 8}
test_data['Functional'] = test_data['Functional'].replace(dic_Functional)

dic_KitchenQual = {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}
test_data['KitchenQual'] = test_data['KitchenQual'].replace(dic_KitchenQual)

dic_LandSlope = {"Sev": 2, "Mod": 1, "Gtl": 3}
test_data['LandSlope'] = test_data['LandSlope'].replace(dic_LandSlope)

dic_LotShape = {"IR3": 1, "IR2": 4, "IR1": 3, "Reg": 2}
test_data['LotShape'] = test_data['LotShape'].replace(dic_LotShape)

dic_PavedDrive = {"N": 0, "P": 1, "Y": 2}
test_data['PavedDrive'] = test_data['PavedDrive'].replace(dic_PavedDrive)

dic_Street = {"Grvl": 1, "Pave": 2}
test_data['Street'] = test_data['Street'].replace(dic_Street)

dic_Utilities = {"ELO": 1, "NoSeWa": 2, "NoSewr": 3, "AllPub": 4}
test_data['Utilities'] = test_data['Utilities'].replace(dic_Utilities)


dic_HeatingQC= {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
dic_Heating={'GasA': 6, 'GasW': 5, 'OthW': 4, 'Wall': 3, 'Grav': 2, 'Floor': 1}
test_data['HeatingQC']= test_data['HeatingQC'].replace(dic_HeatingQC)
test_data['Heating']= test_data['Heating'].replace(dic_Heating)

dic_CentralAir= {"N":0,"Y":1}
test_data["CentralAir"]=test_data['Heating'].replace(dic_CentralAir)

dic_MsZoning= {'C (all)':0,"RM":1,"RH":2,"RL":3,"FV":4 }
test_data["MSZoning"]=test_data["MSZoning"].replace(dic_MsZoning)

dic_LandContour={"Bnk":1,"Lvl":2,"Low":3,"HLS":4 }
test_data["LandContour"]=test_data["LandContour"].replace(dic_LandContour)

dic_Electrical={"Mix":0, "FuseP":1,"FuseF":2,"FuseA":3,"SBrkr":4}
test_data["Electrical"]=test_data["Electrical"].replace(dic_Electrical)

dic_MsVnrType= {"None":0,"BrkCmn":1,"BrkFace":2,"Stone":3}
test_data["MasVnrType"]=test_data["MasVnrType"].replace(dic_MsVnrType)

dic_FireplaceQu= {"None":0, "Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
test_data["FireplaceQu"]=test_data["FireplaceQu"].replace(dic_FireplaceQu)

dic_GarageFinish={"None":0,"Unf":1,"RFn":2,"Fin":3}
test_data["GarageFinish"]=test_data["GarageFinish"].replace(dic_GarageFinish)

#### Hacemos get dummies haciendo que las columnas coincidan con los datos de train

In [34]:
train_data= pd.read_csv("/Users/aitormentxaka/Desktop/Data science/proyectos/Modelo machine learning regresión/src/data/data_clean/train_1.csv")

In [35]:
columnas_categoricas = test_data.select_dtypes(include=['object']).columns.tolist()
test_data_dummies = pd.get_dummies(test_data, columns=columnas_categoricas, dummy_na=True)
test_data_dummies = test_data_dummies.astype(int)

In [36]:
test_data_dummies

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LandSlope,...,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,2,80,11622,2,0,2,2,4,3,...,0,1,0,0,0,0,0,1,0,0
1462,20,3,81,14267,2,0,3,2,4,3,...,0,1,0,0,0,0,0,1,0,0
1463,60,3,74,13830,2,0,3,2,4,3,...,0,1,0,0,0,0,0,1,0,0
1464,60,3,78,9978,2,0,3,2,4,3,...,0,1,0,0,0,0,0,1,0,0
1465,120,3,43,5005,2,0,3,4,4,3,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,1,21,1936,2,0,2,2,4,3,...,0,1,0,0,0,0,0,1,0,0
2916,160,1,21,1894,2,0,2,2,4,3,...,0,1,0,1,0,0,0,0,0,0
2917,20,3,160,20000,2,0,2,2,4,3,...,0,1,0,1,0,0,0,0,0,0


In [37]:

train_data_dummies, test_data_dummies = train_data.iloc[:, :-1].align(test_data_dummies, axis=1, fill_value=0)


In [38]:
train_data_dummies

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,...,SaleType_nan,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,0,3,1,0,0,0,0,...,0,0,2,8,856,4,0,2003,2003,2008
1,1262,0,0,0,3,1,0,0,0,0,...,0,0,2,6,1262,4,298,1976,1976,2007
2,920,866,0,0,3,1,0,0,0,0,...,0,0,2,6,920,4,0,2001,2002,2008
3,961,756,0,0,3,1,0,0,0,0,...,0,0,2,7,756,4,0,1915,1970,2006
4,1145,1053,0,0,4,1,0,0,0,0,...,0,0,2,9,1145,4,192,2000,2000,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,953,694,0,0,3,1,0,0,0,0,...,0,0,2,7,953,4,0,1999,2000,2007
1456,2073,0,0,0,3,1,0,0,0,0,...,0,0,2,7,1542,4,349,1978,1988,2010
1457,1188,1152,0,0,4,1,0,0,0,0,...,0,0,2,9,1152,4,0,1941,2006,2010
1458,1078,0,0,0,2,1,0,0,0,0,...,0,0,2,5,1078,4,366,1950,1996,2010


In [16]:
test_data_dummies

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,...,SaleType_nan,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1461,896,0,0,0,2,1,0,0,0,0,...,0,120,2,5,882,4,140,1961,1961,2010
1462,1329,0,0,0,3,1,0,0,0,0,...,0,0,2,6,1329,4,393,1958,1958,2010
1463,928,701,0,0,3,1,0,0,0,0,...,0,0,2,6,928,4,212,1997,1998,2010
1464,926,678,0,0,3,1,0,0,0,0,...,0,0,2,7,926,4,360,1998,1998,2010
1465,1280,0,0,0,2,0,0,0,0,1,...,0,144,2,5,1280,4,0,1992,1992,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,546,546,0,0,3,0,0,0,1,0,...,0,0,2,5,546,4,0,1970,1970,2006
2916,546,546,0,0,3,0,0,0,0,1,...,0,0,2,6,546,4,0,1970,1970,2006
2917,1224,0,0,0,4,1,0,0,0,0,...,0,0,2,7,1224,4,474,1960,1996,2006


In [17]:
train_data_dummies["SalePrice"]= train_data["SalePrice"]

### Guardamos los datos limpios 

In [42]:

test_data_dummies["Id"]= test_data_dummies.index
train_data_dummies.to_csv('train_data_clean.csv', index=False)
test_data_dummies.to_csv('test_data_clean.csv', index=False)